# Imports

In [6]:
%%capture
import pandas as pd 
import re
from pathlib import Path
from datasets import load_dataset
from importlib import reload
from utils.eval import send_openrouter_request, models


In [14]:
# load transcripts
dir = Path('../data/transcripts')
doc_paths = sorted(dir.glob('*.txt'))

# Eval
"Common sense" checks using openrouter

In [21]:
# create string representing a single conversation
doc = doc_paths[0]

with doc.open('r', encoding = 'utf-8') as f:
    lines = [" ".join(line.split()) for line in f if line.strip()]

In [5]:
# need to think more abt. prompting strategy...lot of variance...
task = '''This transcript contains speaker labels. Some labels refer to the same real person.

Tasks:
1) How many unique real participants are there?
2) Which speaker labels are duplicates of the same person?
3) For each duplicate, say which label it should be merged into.
4) Give one short sentence describing each real participant.'''


In [5]:
messages = [{'role': 'user', 'content': f'{task}: {conversation}'}]

response, reasoning, refusal, provider = send_openrouter_request(messages, model = 'allenai/olmo-3.1-32b-instruct')
print(response)

Here’s a detailed answer to your four tasks based on the provided transcript:

---

### 1) How many unique real participants are there?

There are **three unique real participants** in the transcript:

- **Jasmine** (the interviewer/reporter from NBC News)
- **Speaker 4 / Lynn** (the CEO and co-founder of Fireworks, also referred to as "Lynn" by Jasmine at the end)
- **Speaker 1** (whose identity is not explicitly named, but appears to be someone else in the conversation, possibly a colleague of Jasmine or part of her team)

However, upon close reading, "Speaker 1" is likely the same as Jasmine, based on the context and conversational flow. Jasmine is the one initiating and leading the interview, and "Speaker 1" seems to be her speaking at several points (e.g., "Speaker 1 12:33:04 ... we represent fireworks..."). The transcript labels may be inconsistent, but the content suggests only two main people are speaking before Lynn joins: Jasmine and possibly another person (Speaker 2 and Spe

# Cleaning
Creating a speaker:token mapping


In [38]:
# structure message dictionary
time = re.compile(r'\d{2}:\d{2}:\d{2}')

turns, speaker, msgs = [], None, []

for ln in lines:
    if time.search(ln):
        if speaker and msgs:
            speaker = time.sub('', speaker).strip()
            turns.append({'speaker': speaker, 'text': ' '.join(msgs)})
        speaker, msgs = ln, []
    else:
        if speaker:
            msgs.append(ln)

if speaker and msgs:
    turns.append({"speaker": speaker, "text": " ".join(msgs)})

In [47]:
# decompose to word level
words = []
for t in turns:
    # print(t['text'])
    for w in t['text'].split():
        words.append({'speaker': t['speaker'], 'word': w})

words

[{'speaker': 'PR', 'word': 'Yes,'},
 {'speaker': 'PR', 'word': 'I'},
 {'speaker': 'PR', 'word': 'know'},
 {'speaker': 'PR', 'word': "I'm"},
 {'speaker': 'PR', 'word': 'sorry'},
 {'speaker': 'PR', 'word': 'about'},
 {'speaker': 'PR', 'word': 'all'},
 {'speaker': 'PR', 'word': 'the'},
 {'speaker': 'PR', 'word': 'back'},
 {'speaker': 'PR', 'word': 'and'},
 {'speaker': 'PR', 'word': 'forth,'},
 {'speaker': 'PR', 'word': 'but'},
 {'speaker': 'PR', 'word': 'I'},
 {'speaker': 'PR', 'word': "Lynn's"},
 {'speaker': 'PR', 'word': 'schedule'},
 {'speaker': 'PR', 'word': 'can'},
 {'speaker': 'PR', 'word': 'be'},
 {'speaker': 'PR', 'word': 'crazy,'},
 {'speaker': 'PR', 'word': 'so'},
 {'speaker': 'Jasmine', 'word': 'no'},
 {'speaker': 'Jasmine', 'word': 'worries.'},
 {'speaker': 'Jasmine', 'word': 'I'},
 {'speaker': 'Jasmine', 'word': "can't"},
 {'speaker': 'Jasmine', 'word': 'even'},
 {'speaker': 'Jasmine', 'word': 'imagine'},
 {'speaker': 'PR', 'word': 'CEO'},
 {'speaker': 'PR', 'word': 'things.'